Code to scrap comments from trustpilot website using BeutifulSoup in python

In [ ]:
import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import requests, json
import requests
from lxml.html import fromstring

my_url = 'https://www.trustpilot.com/review/www.bwear.com'

# opening url and grabbing the web page
uClient = urlopen(my_url)
page_html = uClient.read()
uClient.close()


# html parsing
page_soup = soup(page_html, 'html.parser')

filename = "trustpilotReviewss.csv"
f = open(filename, 'w',encoding="utf-8")

headers = "Name, Headline, Reviews\n"

f.write(headers)

#containers = page_soup.findAll('script', {'type':'application/ld+json'})
parsed_data = json.loads(page_soup.find('script',type='application/ld+json').string)
containers = json.dumps(parsed_data,indent=4)
containers = json.loads(containers)
containers=containers[0]['review']

for container in containers:
    name=container['author']['name']
    headline=container['headline']
    reviewBody=container['reviewBody']
    print('Name :',name)
    print('Headline',headline)
    print('Reviews',reviewBody)
    
    f.write(name + ',' + headline + ',' + reviewBody + '\n')

f.close()

#print(containers)




Name : Seven Mile Presbyterian
Headline I love the shirts I have purchased from…
Reviews I love the shirts I have purchased from B-Wear Sportswear.  I get many compliments about them. They come out of the dryer and all I have to do is fold them.
Name : Melinda McDonald
Headline This is my 3 order !
Reviews This is my 3 order !! I love my shirts !! The size fits great and quality is excellent! Thank you !!
Name : Renae Ress
Headline Vey fast and easy transaction
Reviews Vey fast and easy transaction. Also very satisfied with the quality of my order. Thank you B-wear.!!!
Name : Jody
Headline Fun and Fabulous!
Reviews I always love the selection of fun t-shirts. My grown kids love getting them as gifts. Keep up the good work!
Name : Sarah Percival
Headline Great, fun shirts
Reviews Love all the shirts, tank tops and sweatshirts I’ve gotten! The reason for the 3 stars is lack delay in shipment of product, ordered some Halloween shirts and didn’t get them until after Halloween! Yes, they ar

installing google translator library which is used to detect language

In [ ]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 1.3 MB 8.7 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=68dcc0608da6b257d06e441069b116d9b1410a253ed912b9f4c69c25c0613a4a
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


Rule Engine Code.



In [ ]:
pip install langdetect

     |████████████████████████████████| 981 kB 4.4 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=e360b2ba982b9e94fa35401bc5bcaec55c33c646834bebd88d22d067c5a5eb8f
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [ ]:
import pandas as pd
import re
import csv
import urllib.request
from io import StringIO
from langdetect import detect

detector = Translator()

col=['Review']

# Read the csv file into a StringIO object
df = pd.read_csv("https://first-us-east-2-bucket.s3.amazonaws.com/combined-csv-files1.csv")

filename = "FinalOutputReviewStatuss.csv"
f = open(filename, 'w', encoding='utf-8')

headers = "Reviews, Status\n"

f.write(headers)

for review in df['Review']:
    #Calulating charachter Len of comment
    reviewLen = int(len(review))

    #Regex Function to detect 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,review)

    #List of offensive words and Checking if review has any listed offensive word
    offense_services = ["bad", "terrible", "rubbish","insane" ,"garbage"]
    offense_contains_service = review.lower() in (string.lower() for string in offense_services)
    
    #Decoding utf encoded string and detecting the language in review
    Reviews = review.replace('"','')
    Reviews = '"'+Reviews+'"'
    Rev=Reviews.encode('utf-8').decode()
    dec_lan = detect(Rev)
    
    #Conditions flags the reviews having characted length less than 100
    if(reviewLen<100):
            Reviews = Reviews
            Status = 'Flagged'
    #Condition flags the review which has url in it
    elif(len(url)>0):
            Reviews = Reviews
            Status = 'Flagged'
    #Condition flags the review if it has any listed offensive words
    elif(offense_contains_service==True):
            Reviews = Reviews
            Status = 'Flagged'
    
    #Condition flags the review if it's written in any other language than English
    elif(dec_lan != "en"):
            Reviews = Reviews
            Status = 'Flagged'
    #Rest all review will be published
    else:
            Reviews = Reviews
            Status = 'Published'
        
        
    print("Reviews:" + Reviews)
    print("Status:" + Status)
    print(dec_lan)
    
    f.write(Reviews+ ',' + Status + '\n')
    

f.close()